In [1]:
#upload package
import pandas as pd
import numpy as np
import time
import scipy

In [18]:
#upload dataset
train_data = pd.read_csv("D:/EIC-Code/00-Python/Machine-Learning-HW/SVM/bank-note/train.csv", header = None, names = ['variance','skewness','curtosis','entropy','y'])
test_data = pd.read_csv("D:/EIC-Code/00-Python/Machine-Learning-HW/SVM/bank-note/test.csv", names = ['variance','skewness','curtosis','entropy','y'])

features = ['variance','skewness','curtosis','entropy']
outcome = 'y'

X_train = train_data[features].values #change to matrix multiple
y_train = train_data[outcome].values
X_test = test_data[features].values
y_test = test_data[outcome].values
y_train[y_train == 0] = -1
y_test[y_test == 0] = -1

In [79]:
class PrimalSVM:
    def __init__(self, gamma, a, C, N):
        self.gamma = gamma
        self.a = a
        self.C = C
        self.N = N
        self.w = None  # Weight vector will be initialized during training

    def fit(self, X, y, epochs, schedule):
        n_features = X.shape[1]
        self.w = np.zeros(n_features)  # Initialize weights to zeros

        for epoch in range(epochs):
            data = np.column_stack((X, y))
            np.random.shuffle(data)  # Shuffle training data
            X_shuffled = data[:, :-1]
            y_shuffled = data[:, -1]

            for i, (xi, yi) in enumerate(zip(X_shuffled, y_shuffled)):
                t = epoch * len(y_shuffled) + i + 1  # Global step count
                
                # Learning rate schedules
                if schedule == "schedule1":
                    gamma_t = self.gamma / (1 + (self.gamma / self.a) * t)
                elif schedule == "schedule2":
                    gamma_t = self.gamma / (1 + t)
                else:
                    raise ValueError("Invalid schedule. Choose 'schedule1' or 'schedule2'.")

                if yi * np.dot(self.w, xi) <= 1:
                    self.w = self.w - gamma_t * self.w + gamma_t * self.C * self.N * yi * xi
                else:
                    self.w = (1 - gamma_t) * self.w

    def predict(self, X):
        return np.sign(X.dot(self.w))

    def score(self, X, y):
        predictions = self.predict(X)
        errors = np.mean(predictions != y)
        return errors


In [ ]:
np.random.seed(0)
# Define hyperparameter search space
Cs = [100 / 873, 500 / 873, 700 / 873]
gamma_values = [0.1, 0.01, 0.001]#
a_values = [1, 10, 100]
N = 1
epochs = 100


lowest_error = 10000 #float("inf")
best_params = None

for C in Cs:
    for gamma in gamma_values:
        for a in a_values:
            svm = PrimalSVM(gamma=gamma, a=a, C=C, N=N)
            
            svm.fit(X_train, y_train, epochs=epochs, schedule="schedule2")
            train_errors = svm.score(X_train, y_train)
            test_errors = svm.score(X_test, y_test)
            print(f"Testing: C={C}, gamma={gamma}, a={a}, train_error: {train_errors}, test_error:{test_errors}")

            if test_errors < lowest_error:
                lowest_error = test_errors
                best_params = (C, gamma, a)

print(f"Best parameters: C={best_params[0]}, gamma={best_params[1]}, a={best_params[2]}")
print(f"Lowest error: {lowest_error}")

Testing: C=0.1145475372279496, gamma=0.1, a=1, train_error: 0.22477064220183487, test_error:0.218
Testing: C=0.1145475372279496, gamma=0.1, a=10, train_error: 0.18807339449541285, test_error:0.172
Testing: C=0.1145475372279496, gamma=0.1, a=100, train_error: 0.19724770642201836, test_error:0.19
Testing: C=0.1145475372279496, gamma=0.01, a=1, train_error: 0.30275229357798167, test_error:0.304
Testing: C=0.1145475372279496, gamma=0.01, a=10, train_error: 0.3256880733944954, test_error:0.33
Testing: C=0.1145475372279496, gamma=0.01, a=100, train_error: 0.30275229357798167, test_error:0.306
Testing: C=0.1145475372279496, gamma=0.001, a=1, train_error: 0.2798165137614679, test_error:0.286
Testing: C=0.1145475372279496, gamma=0.001, a=10, train_error: 0.3302752293577982, test_error:0.334
Testing: C=0.1145475372279496, gamma=0.001, a=100, train_error: 0.2511467889908257, test_error:0.252
Testing: C=0.572737686139748, gamma=0.1, a=1, train_error: 0.06077981651376147, test_error:0.084
Testing: 

In [93]:
import numpy as np
from scipy.optimize import minimize

class DualSVM:
    def __init__(self, C):
        self.C = C  # Regularization parameter
        self.alpha = None  # Lagrange multipliers
        self.w = None  # Weight vector
        self.b = None  # Bias term


    def fit(self, X, y):
        n_samples, n_features = X.shape
        
        # kernel matrix (linear kernel)
        K = np.dot(X, X.T)

        # define the dual objective function
        def objective(alpha):
            return -np.sum(alpha) + 0.5 * np.sum((alpha * y)[:, None] * (alpha * y) * K)

        # Initial guess for alpha
        alpha0 = np.zeros(n_samples)

        # Bounds for alpha: 0 <= alpha <= C
        bounds = [(0, C) for _ in range(n_samples)]

        # Equality constraint: sum(alpha * y) = 0
        constraints = {
            'type': 'eq',
            'fun': lambda alpha: np.dot(alpha, y),
            'jac': lambda alpha: y
}
        # Solve the optimization problem
        result = minimize(
            objective,
            alpha0,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints
        )

        # extract the optimal alpha
        self.alpha = result.x #minimize the function to get alpha

        # compute weight vector
        self.w = np.sum((self.alpha * y)[:, None] * X, axis=0)

        # compute bias term
        support_vector_idx = np.where((self.alpha > 0) & (self.alpha < self.C))[0][0]
        self.b = y[support_vector_idx] - np.dot(self.w, X[support_vector_idx])

    def predict(self, X):
        return np.sign(np.dot(X, self.w) + self.b)

    def score(self, X, y):
        predictions = self.predict(X)
        return np.mean(predictions != y)


In [94]:
np.random.seed(0)
# Define hyperparameter C
Cs = [100 / 873, 500 / 873, 700 / 873] #, 873

# Train and evaluate the model for different values of C
for C in Cs:
    svm = DualSVM(C)
    svm.fit(X_train, y_train)
    train_error = svm.score(X_train, y_train)
    test_error = svm.score(X_test, y_test)
    print(f"C={C}, Train Accuracy={train_error:.2f}, Test Accuracy={test_error:.2f}")
    print(f"Weights:, {svm.w}, Bias:, {svm.b}")

# Print the weights and bias for the best model
print("Weights:", svm.w)
print("Bias:", svm.b)

d:\EID-Download\02 Academic\Python38\lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


C=0.1145475372279496, Train Accuracy=0.11, Test Accuracy=0.12
Weights:, [-0.94292598 -0.65149184 -0.73372197 -0.04102195], Bias:, 4.14119177534919
C=0.572737686139748, Train Accuracy=0.15, Test Accuracy=0.14
Weights:, [-1.56393784 -1.01405165 -1.18065044 -0.15651687], Bias:, 7.590350666124916
C=0.8018327605956472, Train Accuracy=0.40, Test Accuracy=0.43
Weights:, [-2.04254833 -1.28068891 -1.51351532 -0.24905307], Bias:, 12.975949611428163
Weights: [-2.04254833 -1.28068891 -1.51351532 -0.24905307]
Bias: 12.975949611428163


In [ ]:
import numpy as np
from scipy.optimize import minimize

class GaussianSVM:
    def __init__(self, C, gamma):
        self.C = C
        self.gamma = gamma
        self.alpha = None  # lagrange multipliers
        self.b = None  # bias term
        self.X_train = None  # training features
        self.y_train = None  # training labels

    def gaussian_kernel(self, x1, x2):
        return np.exp(-np.linalg.norm(x1 - x2) ** 2 / self.gamma)

    def kernel_matrix(self, X):
        n_samples = X.shape[0]
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i, j] = self.gaussian_kernel(X[i], X[j])
        return K

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        n_samples, n_features = X.shape

        # Compute the kernel matrix
        K = self.kernel_matrix(X)

        # Define the dual objective function
        def objective(alpha):
            return -np.sum(alpha) + 0.5 * np.sum((alpha * y)[:, None] * (alpha * y) * K)

        # Bounds for alpha: 0 <= alpha <= C
        bounds = [(0, self.C) for _ in range(n_samples)]

        # Equality constraint: sum(alpha * y) = 0
        constraints = {
            'type': 'eq',
            'fun': lambda alpha: np.dot(alpha, y),
            'jac': lambda alpha: y
        }

        # Initial guess for alpha
        alpha0 = np.zeros(n_samples)

        # Solve the optimization problem
        result = minimize(
            objective,
            alpha0,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints,
            options={'maxiter': 1000, 'disp': True}
        )

        # Extract the optimal alpha
        self.alpha = result.x

        # Compute bias term using support vectors
        support_vector_idx = np.where((self.alpha > 1e-4) & (self.alpha < self.C))[0]
        support_vector_idx = support_vector_idx[0]
        self.b = y[support_vector_idx] - np.sum(self.alpha * y * K[support_vector_idx])

    def predict(self, X):
        y_pred = []
        for x in X:
            # Decision function
            decision = np.sum(
                self.alpha * self.y_train *
                np.array([self.gaussian_kernel(x, x_train) for x_train in self.X_train])
            ) + self.b
            y_pred.append(np.sign(decision))
        return np.array(y_pred)

    def score(self, X, y):
        predictions = self.predict(X)
        return np.mean(predictions != y)  # Error rate

    def get_support_vectors(self):
        return np.where((self.alpha > 1e-4) & (self.alpha < self.C))[0]


In [ ]:
np.random.seed(0)
Cs = [100 / 873, 500 / 873, 700 / 873] #, 873
gammas = [0.1, 0.5, 1, 5, 100]

lowest_error = float("inf")
best_params = None

for C in Cs:
    for gamma in gammas:
        gsvm = GaussianSVM(C, gamma)
        gsvm.fit(X_train, y_train)
        train_error = gsvm.score(X_train, y_train)
        test_error = gsvm.score(X_test, y_test)
        print(f"C={C:.4f}, gamma={gamma}, Train error={train_error:.4f}, Test error={test_error:.4f}")

        if test_error < lowest_error:
            lowest_error = test_error
            best_params = (C, gamma)

print(f"Best Parameters: C={best_params[0]:.4f}, gamma={best_params[1]}")
print(f"Best Test Error: {lowest_error:.4f}")

Optimization terminated successfully    (Exit mode 0)
            Current function value: -82.72279493439663
            Iterations: 15
            Function evaluations: 13098
            Gradient evaluations: 15
C=0.1145, gamma=0.1, Train error=0.4461, Test error=0.4420
Optimization terminated successfully    (Exit mode 0)
            Current function value: -74.16551934430456
            Iterations: 23
            Function evaluations: 20084
            Gradient evaluations: 23
C=0.1145, gamma=0.5, Train error=0.4071, Test error=0.4260


In [ ]:
np.random.seed(0)
# Parameters
C = 500 / 873 
gammas = [0.01, 0.1, 0.5, 1, 5]

# Track support vectors
support_vectors = {}

for gamma in gammas:
    gsvm = GaussianSVM(C=C, gamma=gamma)
    gsvm.fit(X_train, y_train)
    sv_indices = gsvm.get_support_vectors()
    support_vectors[gamma] = sv_indices
    print(f"Gamma={gamma}, Number of Support Vectors: {len(sv_indices)}")

# Calculate overlaps between consecutive gammas
for i in range(len(gammas) - 1):
    gamma1, gamma2 = gammas[i], gammas[i + 1]
    overlap = len(np.intersect1d(support_vectors[gamma1], support_vectors[gamma2]))
    print(f"Overlap between gamma={gamma1} and gamma={gamma2}: {overlap}")

Optimization terminated successfully    (Exit mode 0)
            Current function value: -40.666358159348846
            Iterations: 7
            Function evaluations: 707
            Gradient evaluations: 7
Gamma=0.01, Number of Support Vectors: 5
Optimization terminated successfully    (Exit mode 0)
            Current function value: -40.560207048926294
            Iterations: 14
            Function evaluations: 1415
            Gradient evaluations: 14
Gamma=0.1, Number of Support Vectors: 100
Optimization terminated successfully    (Exit mode 0)
            Current function value: -45.44525533458662
            Iterations: 21
            Function evaluations: 2121
            Gradient evaluations: 21
Gamma=0.5, Number of Support Vectors: 97
Optimization terminated successfully    (Exit mode 0)
            Current function value: -48.99569233826322
            Iterations: 24
            Function evaluations: 2424
            Gradient evaluations: 24
Gamma=1, Number of Support Ve

In [ ]:
import numpy as np

class KernelPerceptron:
    def __init__(self, gamma, max_epochs=10):
        self.gamma = gamma
        self.max_epochs = max_epochs
        self.c = None
        self.X_train = None
        self.y_train = None

    def gaussian_kernel(self, x1, x2):
        return np.exp(-np.linalg.norm(x1 - x2)**2 / self.gamma)

    def fit(self, X, y):
        N = X.shape[0]
        self.c = np.zeros(N)  # Mistake counts
        self.X_train = X
        self.y_train = y

        for epoch in range(self.max_epochs):
            for i in range(N):
                # Compute decision function
                decision = sum(self.c[j] * y[j] * self.gaussian_kernel(X[j], X[i]) for j in range(N))
                if y[i] * decision <= 0:  # Misclassified
                    self.c[i] += 1  # Update mistake count

    def predict(self, X):
        y_pred = []
        for x in X:
            # Compute decision function for new point
            decision = sum(self.c[i] * self.y_train[i] * self.gaussian_kernel(self.X_train[i], x) for i in range(len(self.X_train)))
            y_pred.append(np.sign(decision))
        return np.array(y_pred)

    def score(self, X, y):
        predictions = self.predict(X)
        return np.mean(predictions != y)  # Accuracy


In [ ]:
# Generate synthetic data
np.random.seed(0)
# Test different gamma values
gammas = [0.1, 0.5, 1, 5, 100]
for gamma in gammas:
    kp = KernelPerceptron(gamma=gamma)
    kp.fit(X_train, y_train)
    train_error = kp.score(X_train, y_train)
    test_error = kp.score(X_test, y_test)
    print(f"Gamma={gamma}, Train Accuracy={train_error:.4f}, Test Accuracy={test_error:.4f}")
